<a href="https://colab.research.google.com/github/PhotonSpheres/manim_kurs/blob/main/Manim_Tag_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Upadater und ValueTracker


In [ ]:
!sudo apt update
!sudo apt install libcairo2-dev ffmpeg texlive texlive-latex-extra texlive-fonts-extra texlive-latex-recommended texlive-science tipa libpango1.0-dev
!pip install manim
!pip install IPython --upgrade

In [ ]:
from manim import *

Manim Community v0.9.0

Updater gehören zu den eichtigesten Methoden in Manim. Diese erlauben ein Objekt für jedes einzelne Bild der Animation zu verändern. Ein Updater ist immer eine aufrufbare Funktion welche über die Methode `.add_updater` hinzugefügt wird.

In [ ]:
%%manim -v WARNING --progress_bar None --disable_caching -ql NextToUpdater

config.media_width = "50%"

class NextToUpdater(Scene):
    def construct(self):
        dot = Dot(RIGHT*3)

        label = DecimalNumber()
        def dot_position(mobject):
            mobject.set_value(dot.get_center()[0])
            mobject.next_to(dot)
        label.add_updater(dot_position)
        
        self.add(dot, label)
        self.play(
            Rotating(dot, about_point=ORIGIN, angle=TAU, run_time=TAU, rate_func=linear)
        )

Damit ein Updater nicht von der Bildfrequenz abhängt könne wir einen Parameter `dt=1/FPS` verwenden. Diese Updater werden immer für jedes Bild neu aufgerufen.

In [ ]:
%%manim -v WARNING --progress_bar None --disable_caching -ql DtUpdater

config.media_width = "50%"

class DtUpdater(Scene):
    def construct(self):
        line = Square()

        #Let the line rotate 90° per second
        def rotate_90_deg(obj, dt):
            obj.rotate(dt*90*DEGREES)

        
        #line.add_updater(lambda mobject, dt: mobject.rotate(dt*90*DEGREES))

        line.add_updater(rotate_90_deg)

        self.add(line)
        self.wait(4)
        line.remove_updater(rotate_90_deg)
        #line.clear_updaters()
        self.wait(2)

In [ ]:
%%manim -v WARNING --progress_bar None --disable_caching -ql UpdateFunctionWithDt2

config.media_width = "50%"

class UpdateFunctionWithDt2(Scene):
    def construct(self):
        #Se objects
        self.t_offset=0
        orbit=Ellipse(color=GREEN).scale(2.5)
        planet=Dot()

        planet.move_to(orbit.point_from_proportion(0))

        def update_planet(mob,dt):
            rate=dt*0.3
            mob.move_to(orbit.point_from_proportion((self.t_offset + rate)%1))
            self.t_offset += rate
            print(self.t_offset)

        planet.add_updater(update_planet)
        self.add(orbit,planet)
        self.wait(4)
        planet.clear_updaters()
        self.wait(2)

Updater werden häufig mit einem `ValueTracker` verwendet. Dieser erlaubt es eine Zahl im Hintergrund zu speichern und zu verändern.

In [ ]:
%%manim -v WARNING --progress_bar None --disable_caching -ql ValueTrackerExample

config.media_width = "50%"

class ValueTrackerExample(Scene):
    def construct(self):
        number_line = NumberLine(include_numbers=True)
        pointer = Vector(DOWN)
        label = MathTex("x").add_updater(lambda m: m.next_to(pointer, UP))

        tracker = ValueTracker(0)
        pointer.add_updater(
            lambda m: m.next_to(
                        number_line.n2p(tracker.get_value()),
                        UP
                    )
        )
        self.add(number_line, pointer,label)
        self.wait()
        self.play(tracker.animate.set_value(5)),
        self.wait(0.5)
        self.play(tracker.animate.set_value(3))
        self.play(tracker.animate.increment_value(-2))
        self.wait(0.5)
        self.play(tracker.animate.increment_value(-1))
        self.wait()

In [ ]:
%%manim -v WARNING --progress_bar None --disable_caching -ql FunctionTracker

config.media_width = "50%"

class FunctionTracker(Scene):
    def construct(self):
        # f(x) = x**2
        fx = lambda x: x.get_value()**2
        # ValueTrackers definition
        x_value = ValueTracker(0)
        fx_value = ValueTracker(fx(x_value))
        # DecimalNumber definition
        x_tex = DecimalNumber(x_value.get_value()).add_updater(lambda v: v.set_value(x_value.get_value()))
        fx_tex = DecimalNumber(fx_value.get_value()).add_updater(lambda v: v.set_value(fx(x_value)))
        # TeX labels definition
        x_label = MathTex("x = ")
        fx_label = MathTex("x^2 = ")
        # Grouping of labels and numbers
        group = VGroup(x_tex,fx_tex,x_label,fx_label).scale(2.6)
        VGroup(x_tex, fx_tex).arrange(DOWN,buff=3)
        # Align labels and numbers
        x_label.next_to(x_tex,LEFT, buff=0.7,aligned_edge=x_label.get_bottom())
        fx_label.next_to(fx_tex,LEFT, buff=0.7,aligned_edge=fx_label.get_bottom())

        self.add(group.move_to(ORIGIN))
        self.wait(3)
        self.play(
            x_value.animate.set_value(30),
            rate_func=linear,
            run_time=10
        )
        self.wait()
        self.play(
            x_value.animate.set_value(0),
            rate_func=linear,
            run_time=10
        )
        self.wait(3)

In [ ]:
%%manim -v WARNING --progress_bar None --disable_caching -ql TempratureComparison

config.media_width = "50%"

class TempratureComparison(Scene):
    def construct(self):

        cel_tracker = ValueTracker(0)
        cel_label = MathTex("t_1 =")
        cel_number = DecimalNumber(
            cel_tracker.get_value(),
            unit=r"^\circ  \text{C}",
            num_decimal_places=2
        )
        cel_number.next_to(cel_label, RIGHT)
        cel_formula = VGroup(cel_label, cel_number)
        # cel_group = VGroup(cel_label, cel_number).center()

        # cel_updater
        cel_number.add_updater(
            lambda obj: obj.set_value(cel_tracker.get_value())
        )

        # Jetzt gehts um Freihetseinheiten

        fah_label = MathTex("t_2 =")
        fah_number = DecimalNumber(
            0,
            unit = r"^\circ \text{F}"
        )
        fah_number.next_to(fah_label, RIGHT)
        fah_formula = VGroup(fah_label, fah_number)

        # fah_updater
        fah_number.add_updater(
            lambda obj: obj.set_value(cel_tracker.get_value()*(9/5)+32)
        )

        temp_group = VGroup( cel_formula, fah_formula ).arrange(DOWN).center()

        self.add(temp_group)
        self.wait()
        self.play(
            cel_tracker.animate.set_value(20)
        )
        self.wait()

Die Methode `become` kann immer dann benutzt werden sollte man ein Objekt in ein anderes überführen.

In [ ]:
%%manim -v WARNING --progress_bar None --disable_caching -qh MovingDots

config.media_width = "50%"

class MovingDots(Scene):
    def construct(self):
        d1,d2=Dot(color=BLUE),Dot(color=GREEN)
        dg=VGroup(d1,d2).arrange(RIGHT,buff=1)
        l1=Line(d1.get_center(),d2.get_center()).set_color(RED)
        x=ValueTracker(0)
        y=ValueTracker(0)

        self.add(l1,d1,d2)
        self.wait()

        d1.add_updater(lambda d: d.set_x(x.get_value()))
        d2.add_updater(lambda d: d.set_y(y.get_value()))
        l1.add_updater(lambda d: d.become(Line(d1.get_center(),d2.get_center())))
        #self.add(d1,d2)
        self.play(x.animate.set_value(5))
        self.play(y.animate.set_value(4))
        self.wait()

Ebenfalls sehr nützlich kann die Funktion `always_redraw` sein. Diese Funkltion _malt_ das gegebene Objekt neu für jedes Bild.

In [ ]:
%%manim -v WARNING --progress_bar None --disable_caching -ql Tute2

config.media_width = "50%"

class Tute2(Scene):  #ILLUSTRATING POLAR PLANE WITH A SINE CURVE
    def construct(self):
        e = ValueTracker(0.01) #Tracks the end value of both functions

        plane = PolarPlane(radius_max=3).add_coordinates()
        plane.shift(LEFT*2)
        graph1 = always_redraw(lambda : 
        ParametricFunction(lambda t : plane.polar_to_point(2*np.sin(3*t), t), 
        t_range = [0, e.get_value()], color = GREEN)
        )
        dot1 = always_redraw(lambda : Dot(fill_color = GREEN, fill_opacity = 0.8).scale(0.5).move_to(graph1.get_end())
        )

        axes = Axes(x_range = [0, 4, 1], x_length=3, y_range=[-3,3,1], y_length=3).shift(RIGHT*4)
        axes.add_coordinates()
        graph2 = always_redraw(lambda : 
        axes.get_graph(lambda x : 2*np.sin(3*x), x_range = [0, e.get_value()], color = GREEN)
        )
        dot2 = always_redraw(lambda : Dot(fill_color = GREEN, fill_opacity = 0.8).scale(0.5).move_to(graph2.get_end())
        )

        title = MathTex(r"f(\theta) = 2\sin(3\theta)", color = GREEN).next_to(axes, UP, buff=0.2)

        self.play(LaggedStart(
            Write(plane), Create(axes), Write(title),
            run_time=3, lag_ratio=0.5)
        )
        self.add(graph1, graph2, dot1, dot2)
        self.play(e.animate.set_value(PI), run_time = 10, rate_func = linear)
        self.wait()

In [ ]:
%%manim -v WARNING --progress_bar None --disable_caching -ql Tute2

config.media_width = "50%"

class Tute2(Scene):  #ILLUSTRATING POLAR PLANE WITH A SINE CURVE
    def construct(self):

        def butterfly(t):
            arg = np.exp(np.cos(t))-2*np.cos(4*t)-np.sin(t/12)**5
            x = np.sin(t)*arg
            y = np.cos(t)*arg
            return [x,y,0]

        butterfly_graph = ParametricFunction(
            butterfly,
            t_range = [0, 12*PI]
        )

        #self.add(butterfly_graph)
        self.play(
            Create(butterfly_graph),
            run_time=10
        )
        self.wait()